In [1]:
 pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 5.0 MB/s 


In [2]:
import pyodbc
from random import *
import datetime
import pandas as pd

In [3]:
pyodbc.drivers()

[]

In [ ]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=SB2000LAP7316\SQLEXPRESS;'
                      'Database=FRM;'


In [ ]:
cursor = conn.cursor()
dfcursor.execute('select PAYER_MOB from dbo.may order by S_NO')

for i in cursor:
    print(i)

In [ ]:
query = """SELECT PAYER_MOB from dbo.may order by S_NO"""
df = pd.read_sql(query, conn)
print(df)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.rename(columns={'PAYER_ACCT_NO':'Payer Account Number ( for DEBIT)','TRAN_DATE_TIME':'Transaction Date Time','AMOUNT':'Amount','Payee Account Number (CREDIT txn and mobile nuber to fetch from':'Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)','PAYER_ADDRESS':'Payer Address'})
df['Payer Account Number ( for DEBIT)']=pd.to_numeric(df['Payer Account Number ( for DEBIT)'])
df['Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)']=pd.to_numeric(df['Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)'])

In [ ]:
# df.head()

In [ ]:
# df.describe()

In [ ]:
# df.info()

In [ ]:
df1 = df.loc[df['Transaction Type'] == 'DEBIT']
cols = ["Payer Account Number ( for DEBIT)","Transaction Date Time","Amount","Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)", "Payer Address"]
df1 = df1[cols]
df1 = df1.set_index('Payer Account Number ( for DEBIT)')
df1.head()

In [ ]:
def getRecent(sdf):
  try:
    return sdf["Transaction Date Time"].iloc[-1]
  except:
    return sdf["Transaction Date Time"]
def getRecentAcc_no(sdf):
  try:
    return sdf["Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)"].iloc[-1]
  except:
    return sdf["Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)"]
def getAddress(sdf):
  d = dict()
  try:
    max=0
    for i in sdf['Payer Address']:
      upi = i.split('@')[1]
      try:
        d[upi]+=1
        max=max(d[upi],max)
      except:
        d[upi]=1
        max=max(d[upi],max)
  except:
    pass
  d["max_recent_freq"]=max
  return str(d)
def TimeSlot(sdf, start, end):
    ct = 0
    try:
      for i in sdf['Transaction Date Time']:
        time = i.time()
        if time>=start and time<=end:
          ct += 1
    except:
      pass
    return ct
def AmountSlot(sdf, minimum, maximum):
  ct = 0
  try:
    for i in sdf['Amount']:
      if i>=minimum and i<=maximum:
        ct += 1
  except:
    pass
  return ct

In [ ]:
def getData(sdf, unique_acc):
  res = [unique_acc]
  
  maxi=0
  start = datetime.time(0, 0, 0)
  end = datetime.time(3, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(4, 0, 0)
  end = datetime.time(7, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(8, 0, 0)
  end = datetime.time(11, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(12, 0, 0)
  end = datetime.time(15, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(16, 0, 0)
  end = datetime.time(19, 59, 59)
  res.append(TimeSlot(sdf, start, end))
  maxi=max(maxi,TimeSlot(sdf, start, end))

  start = datetime.time(20, 0, 0)
  end = datetime.time(23, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  res.append(maxi)
  maxi=0

  for i in range(0, 100000, 5000):
    minimum = i
    maximum = i+4999
    v=AmountSlot(sdf, minimum, maximum)
    maxi=max(maxi,v)
    res.append(v)

  res.append(maxi)
  address=getAddress(sdf)
  maxpayee=address["max_recent_freq"]
  for key in address:
    address[key]=10.0-((10.0/maxpayee)*address[key])
  del address['max_recent_freq']
  res.append(address)
  res.append(getRecent(sdf))
  res.append(getRecentAcc_no(sdf))

  maxtime=res[7]
  maxamount=res[28]
  for i in range(1,7):
    res[i]=10.0-((10.0/maxtime)*res[i])
  
  for i in range (8,28):
    res[i]=10.0-((10.0/maxamount)*res[i])

  return res

In [ ]:
ct=0
mdf = pd.DataFrame(columns=['Payer Account Number', 'TimeSlot1', 'TimeSlot2', 'TimeSlot3', 'TimeSlot4', 'TimeSlot5', 'TimeSlot6','maxtimefreq', 'Amount1', 'Amount2', 'Amount3', 'Amount4', 'Amount5', 'Amount6', 'Amount7', 'Amount8', 'Amount9', 'Amount10', 'Amount11', 'Amount12', 'Amount13', 'Amount14', 'Amount15', 'Amount16', 'Amount17', 'Amount18', 'Amount19', 'Amount20','maxamountfreq', 'UPI', 'Recent Transaction', 'Last Payee Account Number'])

for unique_acc in df1.index.unique():
  sdf = df1.loc[unique_acc]
  mdf.loc[ct] = getData(sdf, unique_acc)
  ct += 1

In [ ]:
mdf.head()

In [ ]:
def Time(sdf):
  d = dict()
  base = 'TimeSlot'
  times = []
  for i in range(1, 7, 1):
    time = sdf[base+str(i)]
    times.append(time)
  
  times.sort(reverse=True)

  res = []
  
  for i in range(0, 6):
    diff = times[0]-times[i]
    res.append(diff)
  
  return str(res)

In [ ]:
sdf = mdf.iloc[10]
Time(sdf)

In [ ]:
mdf.iloc[10]

In [ ]:
mdf = mdf.set_index('Payer Account Number')

In [ ]:
mdf1=pd.DataFrame(columns=['Payer Account Number', 'TimeSlot1', 'TimeSlot2', 'TimeSlot3', 'TimeSlot4', 'TimeSlot5', 'TimeSlot6','Amount1', 'Amount2', 'Amount3', 'Amount4', 'Amount5', 'Amount6', 'Amount7', 'Amount8', 'Amount9', 'Amount10', 'Amount11', 'Amount12', 'Amount13', 'Amount14', 'Amount15', 'Amount16', 'Amount17', 'Amount18', 'Amount19', 'Amount20', 'UPI', 'Recent Transaction', 'Last Payee Account Number'])

In [ ]:
def get_score(acc_no,amount,payee_acc_num,payer_address):
  payer_address=payer_address.split('@')[1]
  #try and catch block to check whether the payer account number is present in our database or not
  tdf=None
  try:
    tdf=mdf.loc[acc_no]
  except Exception as e:
    print("Account number is not present in the database")
    return 299

  score=0

  
  #To generate score for frequency corresponding to the given payee account number
  last=tdf["Recent Transaction"]
  last=str(last).split('.')[0]
  last=datetime.datetime.strptime(str(last), '%Y-%m-%d %H:%M:%S')
  last_payee=tdf["Last Payee Account Number"]
  now=datetime.datetime.now()
  curr_time=now.time()
  d=now-last
  if d.days==0:
    time_diff=d.seconds//60
    if time_diff<=4 and payee_acc_num==last_payee:
      score=20 
      print("Last transaction was made to the same account number within 4 minutes!!!")
    else:
      score=2.25

  #To generate the score for payer address
  upi=eval(tdf["UPI"])
  if payer_address in upi:
    v=upi.get(payer_address)
    maximum=upi.get('max_recent_freq')
    s=10.0-((10.0/maximum)*v)
    score+=s
  else:
    print("Payer address is not present in the database")
    score+=10.0 #generating very high score because payer address is not present corresponding to the payer account number
  
  #To generate score for time slot
  timeslot="TimeSlot"
  now=datetime.datetime.now()
  curr_time=now.time()
  if curr_time>=datetime.time(0, 0, 0) and curr_time<=datetime.time(3, 59, 59):
    timeslot="TimeSlot1"
  elif curr_time>=datetime.time(4, 0, 0) and curr_time<=datetime.time(7, 59, 59):
    timeslot="TimeSlot2"
  elif curr_time>=datetime.time(8, 0, 0) and curr_time<=datetime.time(11, 59, 59):
    timeslot="TimeSlot3"
  elif curr_time>=datetime.time(12, 0, 0) and curr_time<=datetime.time(15, 59, 59):
    timeslot="TimeSlot4"
  elif curr_time>=datetime.time(16, 0, 0) and curr_time<=datetime.time(19, 59, 59):
    timeslot="TimeSlot5"
  else:
    timeslot="TimeSlot6"
  
  timeslot1=tdf[timeslot]
  maximum=tdf["maxtimefreq"]
  s=10.0-((10.0/maximum)*timeslot1)
  score+=s
  if timeslot1==0:
    print("Out of normal transaction timing habit!!!")
  
  #To generate score for amount slot
  amountslot="Amount"
  k=0
  for i in range(0, 100000, 5000):
    minimum = i
    maximum = i+4999
    k=k+1
    if amount>=minimum and amount<=maximum:
      amountslot=amountslot+str(k)
      break
  amountslot1=tdf[amountslot]
  maximum=tdf["maxamountfreq"]
  s=10.0-((10.0/maximum)*amountslot1)
  score+=s
  if amountslot1==0:
    print("Out of normal transaction amount habit!!!")

  # updating the value of recent transaction date time and payee account number
  tdf["Recent Transaction"]=str(now)
  tdf["Last Payee Account Number"]=payee_acc_num
  mdf.loc[acc_no]=tdf

  return score


In [ ]:
print(get_score(100112250233,250,20148987029,"8130528356@hbd"))

In [ ]:
ab=mdf.loc[100112250233]
print(ab)

In [ ]:
mdf.to_pickle("./dummy.pkl")